[0] Shepp, LA and Logan BF, "The Fourier reconstruction of a head section." IEEE Transactions on Nuclear Science 21, No. 3 (1974)

[1] Toft, PA, "The Radon Transform - Theory and Implementation", PhD dissertation, Departement of Mathematical Modelling, Technical University of Denmark (1996)

[1] is apparently the reference for the higher contrast modified Shepp-Logan phantom.

Below is from here: https://stackoverflow.com/a/32698993

In [1]:
import numpy as np

In [2]:
def section(h, r=1):
    # returns the positive root of intersection of line y = h with circle centered at the origin and radius r
    assert r >= 0
    return np.sqrt(r**2 - h**2) if h < r else 0

In [3]:
def g(x, h, r=1):
    # indefinite integral of circle segment
    # http://www.wolframalpha.com/input/?i=r+*+sin%28acos%28x+%2F+r%29%29+-+h
    return 0.5*(np.sqrt(1 - x**2/r**2)*x*r + r**2*np.arcsin(x/r) - 2*h*x) 

In [4]:
def area2(x0, x1, h, r):
    # area of intersection of an infinitely tall box with left edge at x0, right edge at x1, bottom edge at h and top edge at infinity, with circle centered at the origin with radius r
    if x0 > x1:
        x0, x1 = x1, x0
    s = section(h, r)
    # integrate the area
    return g(max([-s, min([s, x1])]), h, r) - g(max([-s, min([s, x0])]), h, r)

In [5]:
def area1(x0, x1, y0, y1, r):
    if y0 > y1:
        y0, y1 = y1, y0  # this will simplify the reasoning
    if y0 < 0:
        if y1 < 0:
            return area1(x0, x1, -y0, -y1, r)  # the box is completely under, just flip it above and try again 
        else:
            return area1(x0, x1, 0, -y0, r) + area1(x0, x1, 0, y1, r)  # the box is both above and below, divide it to two boxes and go again
    else:
        assert y1 >= 0  # y0 >= 0, which means that y1 >= 0 also (y1 >= y0) because of the swap at the beginning
        return area2(x0, x1, y0, r) - area2(x0, x1, y1, r)  # area of the lower box minus area of the higher box

In [6]:
def area(x0, x1, y0, y1, cx, cy, r):
    x0 -= cx; x1 -= cx
    y0 -= cy; y1 -= cy
    # get rid of the circle center
    return area1(x0, x1, y0, y1, r)

```
3.141593
1.570796
1.570796
1.570796
1.570796
0.785398
0.785398
0.785398
0.785398
1.000000
-0.000000
0.000000
0.000000
0.000000
```

In [7]:
print(area(-10, 10, -10, 10, 0, 0, 1))  # unit circle completely inside a huge box, area of intersection is pi
print(area(-10, 0, -10, 10, 0, 0, 1))   # half of unit circle inside a large box, area of intersection is pi/2
print(area(0, 10, -10, 10, 0, 0, 1))    # half of unit circle inside a large box, area of intersection is pi/2
print(area(-10, 10, -10, 0, 0, 0, 1))   # half of unit circle inside a large box, area of intersection is pi/2
print(area(-10, 10, 0, 10, 0, 0, 1))    # half of unit circle inside a large box, area of intersection is pi/2
print(area(0, 1, 0, 1, 0, 0, 1))        # unit box covering one quadrant of the circle, area of intersection is pi/4
print(area(0, -1, 0, 1, 0, 0, 1))       # unit box covering one quadrant of the circle, area of intersection is pi/4
print(area(0, -1, 0, -1, 0, 0, 1))      # unit box covering one quadrant of the circle, area of intersection is pi/4
print(area(0, 1, 0, -1, 0, 0, 1))       # unit box covering one quadrant of the circle, area of intersection is pi/4
print(area(-.5, .5, -.5, .5, 0, 0, 10)) # unit box completely inside a huge circle, area of intersection is 1 
print(area(-20, -10, -10, 10, 0, 0, 1)) # huge box completely outside a circle (left), area of intersection is 0
print(area(10, 20, -10, 10, 0, 0, 1))   # huge box completely outside a circle (right), area of intersection is 0
print(area(-10, 10, -20, -10, 0, 0, 1)) # huge box completely outside a circle (below), area of intersection is 0
print(area(-10, 10, 10, 20, 0, 0, 1))   # huge box completely outside a circle (above), area of intersection is 0

3.141592653589793
1.5707963267948966
1.5707963267948966
1.5707963267948966
1.5707963267948966
0.7853981633974483
0.7853981633974483
0.7853981633974483
0.7853981633974483
1.0
0.0
0.0
0.0
0.0
